In [ ]:
import polars as pl
import os
from hampel import hampel
from datetime import datetime, timezone
import plotly.express as px

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_level_1_cities_portal.parquet"))

In [ ]:
# Utility

def extract_site_data(df, dates:dict[list[tuple]], site_name:str, plot=False):
    extracted_dates = []
    
    for date in dates[site_name]:
        #print(date[0], date[1], date[2])
        
        df_temp = df.filter(pl.col("system_id") == date[0]).filter(pl.col("creation_timestamp").is_between(date[1], date[2])).collect()
        
        extracted_dates.append(df_temp)
    
    df_extracted = pl.concat(extracted_dates)
    
    if plot:
        fig = px.line(df_extracted, x="creation_timestamp", y="co2", markers=True, title=f"{site_name}: CO2 Corrected [ppm]")
        fig.show()
    else:
        return df_extracted
    
    
def plot_wind_rose(df_w, column:str):
    def find_closest_cardinal_direction(degree: float):
        # Normalize the degree value to be between 0 and 360
        degree = degree % 360

        # Define the cardinal and intermediate directions and their corresponding degrees
        directions = {
            "North": 0,
            "NNE": 22.5,
            "NE": 45,
            "ENE": 67.5,
            "East": 90,
            "ESE": 112.5,
            "SE": 135,
            "SSE": 157.5,
            "South": 180,
            "SSW": 202.5,
            "SW": 225,
            "WSW": 247.5,
            "West": 270,
            "WNW": 292.5,
            "NW": 315,
            "NNW": 337.5,
        }

        # Initialize variables to keep track of the closest direction and its degree difference
        closest_direction = None
        min_difference = float("inf")

        # Iterate over the directions and calculate the difference in degrees
        for direction, direction_degree in directions.items():
            difference = abs(degree - direction_degree)

            # Check if the current difference is smaller than the previous minimum difference
            if difference < min_difference:
                min_difference = difference
                closest_direction = direction

        return directions[closest_direction]

    # create bins for wind direction
    df_w = df_w.filter(pl.col("wd") > 0) \
        .with_columns(pl.col("wd") \
        .map_elements(find_closest_cardinal_direction, return_dtype=float) \
        .alias("cardinal_direction")) \
        .group_by(["cardinal_direction", column]) \
        .len() \
        .sort(column)

    fig = px.bar_polar(
        df_w,
        r="len",
        theta="cardinal_direction",
        color=column,
        template="seaborn",
    )

    fig.add_annotation(text="Calm", x=0.5, y=0.5, showarrow=False, font=dict(size=7))

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="middle",
            y=-0.1,
            xanchor="center",
            x=0.5,
            font=dict(size=14),
        ),
        polar=dict(
            hole=0.1, radialaxis=dict(showticklabels=False, ticks="", linewidth=0)
        ),
        margin=dict(t=110),
        title=dict(
            text=f"Wind Rose: Spike Detection", xanchor="center", yanchor="top"
        ),
    )
    fig.show()

In [ ]:
today = datetime(2024, 8, 30, 0, 0, 0).replace(tzinfo=timezone.utc)

dates = {
      "SENR": [
            (1, datetime(2024, 2, 29, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
      "DLRR": [
            (14, datetime(2023, 11, 22, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 23, 59, 59).replace(tzinfo=timezone.utc)),
            (5, datetime(2024, 2, 28, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "TUMR": [
            (6, datetime(2024, 2, 21, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 5, 11, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (6, datetime(2024, 5, 31, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "RDIR": [
            (2, datetime(2023, 9, 13, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (8, datetime(2024, 3, 15, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "SCHR": [
            (10, datetime(2024, 4, 11, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "FINR": [
            (15, datetime(2023, 11, 16, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (3, datetime(2024, 2, 22, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 4, 2, 23, 59, 59).replace(tzinfo=timezone.utc)),
            (11, datetime(2024, 4, 11, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "TAUR": [
            (8, datetime(2023, 10, 27, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (12, datetime(2024, 2, 14, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "FELR": [
            (7, datetime(2023, 12, 14, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (13, datetime(2024, 2, 22, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "SWMR": [
            (15, datetime(2024, 6, 14, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "MAIR": [
            (1, datetime(2023, 9, 8, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2023, 12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (16, datetime(2024, 2, 8, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "PASR": [
            (5, datetime(2023, 11, 16, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 2, 6, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (18, datetime(2024, 2, 8, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "GROR": [
            (4, datetime(2023, 9, 22, 0, 0, 0).replace(tzinfo=timezone.utc), datetime(2024, 2, 12, 0, 0, 0).replace(tzinfo=timezone.utc)),
            (20, datetime(2024, 2, 14, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ],
      "BLUT_48": [
            (14, datetime(2024, 6, 23, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
      "BLUT_85": [
            (7, datetime(2024, 6, 23, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
      "NPLR": [
            (9, datetime(2024, 6, 26, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
      "BOGR": [
            (17, datetime(2024, 7, 9, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
      "HARR": [
            (4, datetime(2024, 7, 30, 0, 0, 0).replace(tzinfo=timezone.utc), today)
            ], 
 }

In [ ]:
# SENR Klenze-Gymnasium Sendling
df_site = extract_site_data(df=df, dates = dates, site_name="SENR")

plot_wind_rose(df_site, column="Flag")

![](sites/SENR.jpg)

In [ ]:
# DLRR DLR Oberpfaffenhofen
df_site = extract_site_data(df=df, dates = dates, site_name="DLRR")

plot_wind_rose(df_site, column="Flag")

![](sites/DLRR.jpg)

In [ ]:
# TUMR TUM Zentralgelände Nord Maxvorstadt
df_site = extract_site_data(df=df, dates = dates, site_name="TUMR")

plot_wind_rose(df_site, column="Flag")

![](sites/TUMR.jpg)

In [ ]:
# RDIR TUM Klinikum rechts der Isar Haidhausen
df_site = extract_site_data(df=df, dates = dates, site_name="RDIR")

plot_wind_rose(df_site, column="Flag")

![picture](sites/RDIR.jpg)

In [19]:
# SCHR Städt. Willi-Graf-Gymnasium Schwabing-West
df_site = extract_site_data(df=df, dates = dates, site_name="SCHR")

plot_wind_rose(df_site, column="Flag")

# No real spikes visible

![](sites/SCHR.jpg)

In [20]:
# FINR Rathaus Gemeinde Finsing
df_site = extract_site_data(df=df, dates = dates, site_name="FINR")

plot_wind_rose(df_site, column="Flag")

![picture](sites/FINR.jpg)

In [21]:
# TAUR Walter-Klingenbeck-Realschule Taufkirchen
df_site = extract_site_data(df=df, dates = dates, site_name="TAUR")

plot_wind_rose(df_site, column="Flag")

![picture](sites/TAUR.jpg)

In [ ]:
# FELR Rathaus Gemeinde Feldkirchen
df_site = extract_site_data(df=df, dates = dates, site_name="FELR")

plot_wind_rose(df_site, column="Flag")

![p](sites/FELR.jpg)

In [ ]:
# SWMR Stadtwerke München Zentrale Moosach
df_site = extract_site_data(df=df, dates = dates, site_name="SWMR")

plot_wind_rose(df_site, column="Flag")

![](sites/SWMR.jpg)

In [ ]:
# MAIR Mittelschule Maisach
df_site = extract_site_data(df=df, dates = dates, site_name="MAIR")

plot_wind_rose(df_site, column="Flag")

![picture](sites/MAIR.jpg)

In [ ]:
# PASR Städt. Bertolt-Brecht-Gymnasium Pasing
df_site = extract_site_data(df=df, dates = dates, site_name="PASR")

plot_wind_rose(df_site, column="Flag")

![p](sites/PASR.jpg)

In [ ]:
# GROR LMU Klinikum Campus Großhadern
df_site = extract_site_data(df=df, dates = dates, site_name="GROR")

plot_wind_rose(df_site, column="Flag")

![](sites/GROR.jpg)

In [ ]:
# NPLR München Klinik Neuperlach
df_site = extract_site_data(df=df, dates = dates, site_name="NPLR")

plot_wind_rose(df_site, column="Flag")

![](sites/NPLR.jpg)

In [ ]:
# BOGR München Klinik Bogenhausen

df_site = extract_site_data(df=df, dates = dates, site_name="BOGR")

plot_wind_rose(df_site, column="Flag")

![](sites/BOGR.jpg)

In [ ]:
# HARR München Klinik Harlaching

df_site = extract_site_data(df=df, dates = dates, site_name="HARR")

plot_wind_rose(df_site, column="Flag")

![](sites/HARR.jpg)